# 特徴選択

Scikit-Learnにステップワイズ法はないが、再帰的に特徴量を消去していく sklearn.feature_selection.RFE があるため、これを用いて特徴選択を行う。

- https://scikit-learn.org/stable/modules/feature_selection.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

### RFE

アルゴリズム
- すべての説明変数によるモデルを作り、最も重要度の低い説明変数を削除
- 残った説明変数によるモデルを作り、最も重要度の低い説明変数を削除
- 指定の数の説明変数になるまで繰り返す

説明変数の重要度
- coef_: 係数
- feature_importances_: 情報利得

In [ ]:
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')

# 2値データへの分類のため、species から setosa を除外
# speciesを 0, 1 にするためにダミー変数化
iris_dummies = pd.get_dummies(data=iris.query('species!="setosa"'), drop_first=True)
# 確認
iris_dummies.iloc[[0, 50]]

### ロジスティック回帰モデルを用いて特徴選択

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

Y_label = 'species_virginica'
X = iris_dummies.drop([Y_label], axis=1)
Y = iris_dummies[Y_label]

# ロジスティック回帰モデルを用いて特徴選択
selector = RFE(estimator=LogisticRegression(C=10000.0), n_features_to_select=2)
# 特徴選択を実行
selector.fit(X, Y)
print(selector.support_)
print(selector.ranking_)

In [ ]:
# 削除後のDataFrame
X_selected_LR = pd.DataFrame(selector.transform(X), columns=X.columns.values[selector.support_])
# 確認
X_selected_LR

### 決定木を用いて特徴選択

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 決定木を用いて特徴選択
selector = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=2)
# 特徴選択を実行
selector.fit(X, Y)
# 削除後のDataFrame
X_selected_DT = pd.DataFrame(selector.transform(X), columns=X.columns.values[selector.support_])
# ranking_ の表示
print(selector.ranking_)
# 確認
X_selected_DT

### ロジスティック回帰による分類

In [ ]:
from sklearn.metrics import roc_curve, auc

def Logistic_Model(X, Y):
    """
    ロジスティック回帰モデルを作成
    """
    model = LogisticRegression(C=10000.0, max_iter=10000000)
    model.fit(X, Y)
    return model

def calc_AUC(model, X, Y):
    """
    AUCの算出
    """
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    return auc(fpr, tpr)

- ロジスティック回帰モデルによる特徴選択の結果

In [ ]:
model = Logistic_Model(X_selected_LR, Y)
AUC = calc_AUC(model, X_selected_LR, Y)
print('AUC={:.3f}'.format(AUC))

- 決定木による特徴選択の結果

In [ ]:
model = Logistic_Model(X_selected_DT, Y)
AUC = calc_AUC(model, X_selected_DT, Y)
print('AUC={:.3f}'.format(AUC))